In [10]:
import iris
import iris.coord_categorisation
import matplotlib.pyplot as plt
import numpy as np
import iris.quickplot as qplt
import netCDF4
import datetime
import scipy
import scipy.signal
import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from scipy.stats import t
import pickle
import os


"""
Potential issues to think about:
    William uses: Might only SST values (because from satilite). Models are at best deaily mean
    They use a standardising year of 1988.3 which would not necessarily be a sensble year to use given model metholology
    


Skirving methodology
Regarding the current methodology for deriving the MMM it goes like this:



1)  for the period of 1985 to 2012, turn each yearly month into an average SST value (only night-only SST values).
2)  for each month, fit a linear regression to the 28 SST values across all years  ie Y = a + bX, where Y is the monthly averaged SST and X is the year
3)  at this stage you will have 12 linear regressions per pixel, one for each month.  Determine the SST value for X = 1988.3
4)  this will provide you with 12 SST values, the maximum of which will be the MMM value for that pixel, the month that this value represents is called the MMM month.
"""

directory = '/data/dataSSD1/ph290/three_hourly/'

In [11]:
def extract_region(cube,lon_west,lon_east,lat_south,lat_north):
    cube_region_tmp = cube.intersection(longitude=(lon_west, lon_east))
    cube_region = cube_region_tmp.intersection(latitude=(lat_south, lat_north))
    return cube_region

In [12]:
def area_avg(cube):
    try:
        cube.coord('latitude').guess_bounds()
        cube.coord('longitude').guess_bounds()
    except:
        pass
    grid_areas = iris.analysis.cartography.area_weights(cube)
    return cube.collapsed(['longitude','latitude'],iris.analysis.MEAN, weights=grid_areas)

In [13]:

def linregress_3D(y_array):
    # y_array is a 3-D array formatted like (time,lon,lat)
    # The purpose of this function is to do linear regression using time series of data over each (lon,lat) grid box with consideration of ignoring np.nan
    # Construct x_array indicating time indexes of y_array, namely the independent variable.
    x_array=np.empty(y_array.shape)
    for i in range(y_array.shape[0]): x_array[i,:,:]=i+1 # This would be fine if time series is not too long. Or we can use i+yr (e.g. 2019).
    x_array[np.isnan(y_array)]=np.nan
    # Compute the number of non-nan over each (lon,lat) grid box.
    n=np.sum(~np.isnan(x_array),axis=0)
    # Compute mean and standard deviation of time series of x_array and y_array over each (lon,lat) grid box.
    x_mean=np.nanmean(x_array,axis=0)
    y_mean=np.nanmean(y_array,axis=0)
    x_std=np.nanstd(x_array,axis=0)
    y_std=np.nanstd(y_array,axis=0)
    # Compute co-variance between time series of x_array and y_array over each (lon,lat) grid box.
    cov=np.nansum((x_array-x_mean)*(y_array-y_mean),axis=0)/n
    # Compute correlation coefficients between time series of x_array and y_array over each (lon,lat) grid box.
    cor=cov/(x_std*y_std)
    # Compute slope between time series of x_array and y_array over each (lon,lat) grid box.
    slope=cov/(x_std**2)
    # Compute intercept between time series of x_array and y_array over each (lon,lat) grid box.
    intercept=y_mean-x_mean*slope
    # Compute tstats, stderr, and p_val between time series of x_array and y_array over each (lon,lat) grid box.
    tstats=cor*np.sqrt(n-2)/np.sqrt(1-cor**2)
    stderr=slope/tstats
    p_val=t.sf(tstats,n-2)*2
    # Compute r_square and rmse between time series of x_array and y_array over each (lon,lat) grid box.
    # r_square also equals to cor**2 in 1-variable lineare regression analysis, which can be used for checking.
    r_square=np.nansum((slope*x_array+intercept-y_mean)**2,axis=0)/np.nansum((y_array-y_mean)**2,axis=0)
    rmse=np.sqrt(np.nansum((y_array-slope*x_array-intercept)**2,axis=0)/n)
    # Do further filteration if needed (e.g. We stipulate at least 3 data records are needed to do regression analysis) and return values
    n=n*1.0 # convert n from integer to float to enable later use of np.nan
    n[n<3]=np.nan
    slope[np.isnan(n)]=np.nan
    intercept[np.isnan(n)]=np.nan
    p_val[np.isnan(n)]=np.nan
    r_square[np.isnan(n)]=np.nan
    rmse[np.isnan(n)]=np.nan
#     return n,slope,intercept,p_val,r_square,rmse
    return slope,intercept

In [14]:
# https://rmets.onlinelibrary.wiley.com/doi/pdf/10.1002/joc.3486
# the monthly data set for 1982 – 2006 was first detrended using a linear regression, calculated for each month of the year and grid cell. The data set was detrended and centred on 1988
# I don't like this - it becoes ver deending on whether 1988 is a warm or cold year/...

def mmm(cube,years_for_mmm_climatology):
    cube_years = cube.coord('year').points
    #subset the data into the bit you want to use to calculate the MMM climatology and the bit you want to calculate DHW on
    clim_cube = cube[np.where((cube_years > years_for_mmm_climatology[0]) & (cube_years < years_for_mmm_climatology[1]))]
    #collapse the months together, taking the maximum value at each lat-lon grid square
    mmm_climatology = clim_cube.collapsed('time',iris.analysis.MAX)
    return mmm_climatology

def mmm_with_detrending(cube,years_for_mmm_climatology):
    cube_years = cube.coord('year').points
    #subset the data into the bit you want to use to calculate the MMM climatology and the bit you want to calculate DHW on
    clim_cube = cube[np.where((cube_years > years_for_mmm_climatology[0]) & (cube_years < years_for_mmm_climatology[1]))]
    monthly_mean_climatology = clim_cube.collapsed('time',iris.analysis.MEAN)
    clim_cube_detrended = clim_cube.copy()
    clim_cube_detrended.data = scipy.signal.detrend(clim_cube_detrended.data,axis = 0)
    clim_cube_final = clim_cube_detrended + monthly_mean_climatology
    #collapse the months together, taking the maximum value at each lat-lon grid square
    mmm_climatology = clim_cube_final.collapsed('time',iris.analysis.MAX)
    return mmm_climatology


def mmm_skirving(cube):
    print 'NOTE THIS SHOULD BE USING NIGHT TIME TEMPERATURES'
    years_for_mmm_climatology = [1985,2012]
    standardisation_date = 1988.3
    mm_cube = cube[0:12].copy()
    mm_cube_data = mm_cube.data.copy()
    cube_years = cube.coord('year').points
    #subset the data into the bit you want to use to calculate the MMM climatology and the bit you want to calculate DHW on
    clim_cube = cube[np.where((cube_years >= years_for_mmm_climatology[0]) & (cube_years <= years_for_mmm_climatology[1]))]
    clim_cube_detrended = clim_cube.copy()
    clim_cube_detrended_data = clim_cube_detrended.data
    print np.shape(clim_cube_detrended)
    for i,month in enumerate(np.unique(cube.coord('month_number').points)):
        loc = np.where(clim_cube.coord('month_number').points == month)
        tmp = clim_cube_detrended_data[loc,:,:].data[0]
        tmp[np.where(tmp > 1.0e19 )] = np.nan
        slope,intercept = linregress_3D(tmp)
        x = standardisation_date - years_for_mmm_climatology[0]
        y = (slope * x ) + intercept
        mm_cube_data[i,:,:] = y
    mm_cube.data = mm_cube_data
    mmm_climatology = mm_cube.collapsed('time',iris.analysis.MAX)
    return mmm_climatology

In [15]:


def dhm(cube,mmm_climatology,years_over_which_to_calculate_dhw):
    cube_years = cube.coord('year').points
    main_cube = cube[np.where((cube_years > years_over_which_to_calculate_dhw[0]) & (cube_years < years_over_which_to_calculate_dhw[1]))]
    #subtract the monthly mean climatology from the rest of the data
    main_cube -= mmm_climatology

    #set all values less than 1 to zero
    main_cube.data[np.where(main_cube.data <= 1.0)] = 0.0
    #OR
#     main_cube.data[np.where(main_cube.data <= 0.0)] = 0.0

    #make a cube to hold the output data
    output_cube = main_cube[2::].copy()
    output_cube.data[:] = np.nan
    output_cube_data = output_cube.data.copy()

    #loop through from day 84 to the end of the dataset
    for i in range(output_cube.shape[0]):
#         print i,' of ',output_cube.shape[0]
        #sum the temperatures in that 84 day window and divide result by 7 to get in DHWeeks rather than DHdays
        tmp_data = main_cube[i:i+3].collapsed('time',iris.analysis.SUM)
        output_cube_data[i,:,:] = tmp_data.data

    #save the output
    output_cube.data = output_cube_data
    return output_cube



def dhm_Spillman_2013(cube,mmm_climatology,years_over_which_to_calculate_dhw):
    cube_years = cube.coord('year').points
    main_cube = cube[np.where((cube_years > years_over_which_to_calculate_dhw[0]) & (cube_years < years_over_which_to_calculate_dhw[1]))]
    #subtract the monthly mean climatology from the rest of the data
    main_cube -= mmm_climatology

    #set all values less than 1 to zero
#     main_cube.data[np.where(main_cube.data <= 1.0)] = 0.0
    #OR
    main_cube.data[np.where(main_cube.data <= 0.0)] = 0.0

    #make a cube to hold the output data
    output_cube = main_cube[2::].copy()
    output_cube.data[:] = np.nan
    output_cube_data = output_cube.data.copy()

    #loop through from day 84 to the end of the dataset
    for i in range(output_cube.shape[0]):
#         print i,' of ',output_cube.shape[0]
        #sum the temperatures in that 84 day window and divide result by 7 to get in DHWeeks rather than DHdays
        tmp_data = main_cube[i:i+3].collapsed('time',iris.analysis.SUM)
        output_cube_data[i,:,:] = tmp_data.data

    #save the output
    output_cube.data = output_cube_data
    return output_cube

In [16]:
def dhw(cube,mmm_climatology,years_over_which_to_calculate_dhw):
    cube_years = cube.coord('year').points
    #note this is to be uef with daily data...
    main_cube = cube[np.where((cube_years > years_over_which_to_calculate_dhw[0]) & (cube_years < years_over_which_to_calculate_dhw[1]))]
    #subtract the monthly mean climatology from the rest of the data
    main_cube -= mmm_climatology

    #set all values less than 1 to zero
    main_cube.data[np.where(main_cube.data <= 1.0)] = 0.0

    #make a cube to hold the output data
    output_cube = main_cube[83::].copy()
    output_cube.data[:] = np.nan
    output_cube_data = output_cube.data.copy()

    #loop through from day 84 to the end of the dataset
    for i in range(output_cube.shape[0]):
#         print i,' of ',output_cube.shape[0]
        #sum the temperatures in that 84 day window and divide result by 7 to get in DHWeeks rather than DHdays
        tmp_data = main_cube[i:i+3].collapsed('time',iris.analysis.SUM)/7
        output_cube_data[i,:,:] = tmp_data.data

    #save the output
    output_cube.data = output_cube_data
    return output_cube

In [17]:
def merge_two_cubes_extract_midnight(cubes):
    #extract data for midnight
    try:
        iris.coord_categorisation.add_hour(cubes[0], 'time', name='hour')
    except:
        pass
    try:
        iris.coord_categorisation.add_hour(cubes[1], 'time', name='hour')
    except:
        pass
    cube_tmp1 = cubes[0][np.where(cubes[0].coord('hour').points == 0)]
    cube_tmp2 = cubes[1][np.where(cubes[1].coord('hour').points == 0)]
                    
    data1 = cube_tmp1.data
    data2 = cube_tmp2.data
    data = np.concatenate([data1,data2],axis=0)
    data = np.ma.masked_where(data == cubes[0].data.fill_value, data)

    length = cube_tmp1.shape[0] + cube_tmp2.shape[0]
    datetime_object1 = netCDF4.num2date(cube_tmp1.coord('time').points,cube_tmp1.coord('time').units.name,cube_tmp1.coord('time').units.calendar)
    datetime_object2 = netCDF4.num2date(cube_tmp2.coord('time').points,cube_tmp12.coord('time').units.name,cube_tmp2.coord('time').units.calendar)
    datetime_object = np.concatenate([datetime_object1,datetime_object2])
    try:
        tmp =  [x._to_real_datetime() - datetime.datetime(1850,1,1) for x in datetime_object]
    except:
        tmp =  [x - datetime.datetime(1850,1,1) for x in datetime_object]
    days_since_18500101 = [x.days for x in tmp]

    time = iris.coords.DimCoord(days_since_18500101, standard_name='time',long_name=u'time', var_name='time', units='days since 1850-1-1')
    latitude = iris.coords.DimCoord(range(-90, 90, 1), standard_name='latitude', units='degrees')
    longitude = iris.coords.DimCoord(range(0, 360, 1), standard_name='longitude', units='degrees')
    cube = iris.cube.Cube(data,standard_name='sea_surface_temperature',long_name='Sea Surface Temperature', var_name='tos', units='K',dim_coords_and_dims=[(time,0), (latitude, 1),
    (longitude, 2)])
    iris.coord_categorisation.add_year(cube, 'time', name='year')
    iris.coord_categorisation.add_month(cube, 'time', name='month')
    iris.coord_categorisation.add_month_number(cube, 'time', name='month_number')
    return cube


def merge_two_cubes_daily_average(cubes):
    #calculate daily mean from 3 hour snapshots
    try:
        iris.coord_categorisation.add_day_of_year(cubes[0], 'time', name='day')
    except:
        pass
    try:
        iris.coord_categorisation.add_day_of_year(cubes[1], 'time', name='day')
    except:
        pass

    try:
        iris.coord_categorisation.add_year(cubes[0], 'time', name='year')
    except:
        pass
    try:
        iris.coord_categorisation.add_year(cubes[1], 'time', name='year')
    except:
        pass
    
    cubes[0] = cubes[0].aggregated_by(['year','day'], iris.analysis.MEAN)
    cubes[1] = cubes[1].aggregated_by(['year','day'], iris.analysis.MEAN)
    
    data1 = cubes[0].data
    data2 = cubes[1].data
    data = np.concatenate([data1,data2],axis=0)
    data = np.ma.masked_where(data == cubes[0].data.fill_value, data)

    length = cubes[0].shape[0] + cubes[1].shape[0]
    datetime_object1 = netCDF4.num2date(cubes[0].coord('time').points,cubes[0].coord('time').units.name,cubes[0].coord('time').units.calendar)
    datetime_object2 = netCDF4.num2date(cubes[1].coord('time').points,cubes[1].coord('time').units.name,cubes[1].coord('time').units.calendar)
    datetime_object = np.concatenate([datetime_object1,datetime_object2])
    try:
        tmp =  [x._to_real_datetime() - datetime.datetime(1850,1,1) for x in datetime_object]
    except:
        tmp =  [x - datetime.datetime(1850,1,1) for x in datetime_object]
    days_since_18500101 = [x.days for x in tmp]

    time = iris.coords.DimCoord(days_since_18500101, standard_name='time',long_name=u'time', var_name='time', units='days since 1850-1-1')
    latitude = iris.coords.DimCoord(range(-90, 90, 1), standard_name='latitude', units='degrees')
    longitude = iris.coords.DimCoord(range(0, 360, 1), standard_name='longitude', units='degrees')
    cube = iris.cube.Cube(data,standard_name='sea_surface_temperature',long_name='Sea Surface Temperature', var_name='tos', units='K',dim_coords_and_dims=[(time,0), (latitude, 1),
    (longitude, 2)])
    iris.coord_categorisation.add_year(cube, 'time', name='year')
    iris.coord_categorisation.add_month(cube, 'time', name='month')
    iris.coord_categorisation.add_month_number(cube, 'time', name='month_number')
    return cube

In [23]:
def merge_two_cubes_extract_midnight2(cube):
    #extract data for midnight
    try:
        iris.coord_categorisation.add_hour(cube, 'time', name='hour')
    except:
        pass

    cube = cube[np.where(cube.coord('hour').points == 0)]
                    
    cube.data = np.ma.masked_where(cube.data == cube.data.fill_value, cube.data)

    iris.coord_categorisation.add_month(cube, 'time', name='month')
    iris.coord_categorisation.add_month_number(cube, 'time', name='month_number')
    return cube


def merge_two_cubes_daily_average2(cube):
    #calculate daily mean from 3 hour snapshots
    try:
        iris.coord_categorisation.add_day_of_year(cube, 'time', name='day')
    except:
        pass
    try:
        iris.coord_categorisation.add_year(cube, 'time', name='year')
    except:
        pass
    
    cube = cube.aggregated_by(['year','day'], iris.analysis.MEAN)
    
    cube.data = np.ma.masked_where(cube.data == cube.data.fill_value, cube.data)

    iris.coord_categorisation.add_month(cube, 'time', name='month')
    iris.coord_categorisation.add_month_number(cube, 'time', name='month_number')
    return cube

set up the dictonary with model names

In [18]:
models = ['MPI-ESM1-2-HR']

colors = ['r','g','b','k','m','c']
model_dict = {}
for i,model in enumerate(models):
    model_dict[model] = {}
    model_dict[model]['color'] = colors[i]


# tos_3hr_MPI-ESM1-2-HR_historical_r1i1p1f1_gn_regridded.nc

read in the model data

In [19]:
test = os.path.isfile('/data/dataSSD1/ph290/three_hourly/dhw_test.pickle')

if test:
    in_pickle = open('/data/dataSSD1/ph290/three_hourly/dhw_test.pickle', 'r')
    cube_dict = pickle.load(in_pickle)
    in_pickle.close()
else:
    cube_dict = {}

    for model in models:
        cube_dict[model] = {}
        # file = glob.glob(model+'_tos_*_r1i1p1f1_gn.nc')
        # print file

        cubes = iris.load_cube(directory+'tos_3hr_'+model+'_*r1i1p1f1_gn_regridded.nc')
        cube= merge_two_cubes_extract_midnight2(cubes)
        cube_dict[model]['cube_midnight'] = cube
        
        cubes = iris.load_cube(directory+'tos_3hr_'+model+'_*r1i1p1f1_gn_regridded.nc')
        cube2 = merge_two_cubes_daily_average2(cubes)                                   
        cube_dict[model]['cube_daily_avg'] = cube2
    
    out = open('/data/dataSSD1/ph290/three_hourly/dhw_test.pickle', 'w')
    pickle.dump(cube_dict, out )
    out.close()
    

/home/ph290/anaconda2/lib/python2.7/site-packages/iris/fileformats/cf.py:798: UserWarning: Missing CF-netCDF measure variable u'areacello', referenced by netCDF variable u'tos'
  warnings.warn(message % (variable_name, nc_var_name))


MemoryError: Failed to realise the lazy data as there was not enough memory available.
The data shape would have been (248368, 180, 360) with dtype('float32').
 Consider freeing up variables or indexing the data before trying again.

In [22]:
print cube_dict[model]['cube_midnight'][0].data
print cube_dict[model]['cube_daily_avg'][0].data

[[-- -- -- ... -- -- --]
 [-- -- -- ... -- -- --]
 [-- -- -- ... -- -- --]
 ...
 [-1.899847388267517 -1.899835467338562 -1.899824857711792 ...
  -1.8998810052871704 -1.8998701572418213 -1.8998589515686035]
 [-1.8999125957489014 -1.8999110460281372 -1.8999093770980835 ...
  -1.8998948335647583 -1.8999065160751343 -1.899914264678955]
 [-1.8997936248779297 -1.8997925519943237 -1.8997913599014282 ...
  -1.8997972011566162 -1.8997960090637207 -1.8997948169708252]]


KeyError: 'cube_daily_avg'

In [ ]:
cubes = iris.load(directory+'tos_3hr_'+model+'_*r1i1p1f1_gn_regridded.nc')[0]
cubes = cubes[0:10]
#extract data for midnight
try:
    iris.coord_categorisation.add_hour(cubes, 'time', name='hour')
except:
    pass
test = cubes[np.where(cubes.coord('hour').points == 0)]

In [ ]:
test2 = cubes.copy()

#calculate daily mean from 3 hour snapshots
try:
    iris.coord_categorisation.add_day_of_year(test2, 'time', name='day')
except:
    pass

try:
    iris.coord_categorisation.add_year(test2, 'time', name='year')
except:
    pass

test2 = test2.aggregated_by(['year','day'], iris.analysis.MEAN)

In [ ]:
print test[0].data
print test2[0].data

# Calculate DHW

In [ ]:
for model in models:
    print model
    years_over_which_to_calculate_dhw = [2012,2100]
    mmm_climatology = mmm_skirving(cube_dict[model]['cube_midnight'])
    dhm_cube = dhw(cube_dict[model]['cube_midnight'],mmm_climatology,years_over_which_to_calculate_dhw)
#     dhm_cube_Spillman_2013 = dhm_Spillman_2013(cube_dict[model]['cube_midnight'],mmm_climatology,years_over_which_to_calculate_dhw)
    lon_west = 142.0
    lon_east = 157.0
    lat_south = -30.0
    lat_north = -10.0
    dhm_cube_gbr = extract_region(dhm_cube,lon_west,lon_east,lat_south,lat_north)
    cube_dict[model]['dhw_midnight'] = dhm_cube_gbr
    

In [ ]:
for model in models:
    print model
    mmm_climatology = mmm_skirving(cube_dict[model]['cube_daily_avg'])
    dhm_cube = dhw(cube_dict[model]['cube_daily_avg'],mmm_climatology,years_over_which_to_calculate_dhw)
#     dhm_cube_Spillman_2013 = dhm_Spillman_2013(cube_dict[model]['cube_midnight'],mmm_climatology,years_over_which_to_calculate_dhw)
    lon_west = 142.0
    lon_east = 157.0
    lat_south = -30.0
    lat_north = -10.0
    dhm_cube_gbr = extract_region(dhm_cube,lon_west,lon_east,lat_south,lat_north)
    cube_dict[model]['dhw_daily_avg'] = dhm_cube_gbr
#     cube_dict[model]['dhm_cube_Spillman_2013'] = dhm_cube_Spillman_2013
    

In [ ]:
print np.max(cube_dict[model]['cube_daily_avg'][200].data)
print np.max(cube_dict[model]['cube_midnight'][200].data)

In [ ]:
def asb(cube,threshold):
    dhm_cube_gbr_tmp = cube.copy()
    dhm_cube_gbr_tmp_data = dhm_cube_gbr_tmp.data
    dhm_cube_gbr_tmp_data[np.where(dhm_cube_gbr_tmp_data <= threshold)] = 0.0
    dhm_cube_gbr_tmp_data[np.where(dhm_cube_gbr_tmp_data > threshold)] = 1.0
    dhm_cube_gbr_tmp.data = dhm_cube_gbr_tmp_data
    dhm_cube_gbr_asb = dhm_cube_gbr.copy()
    dhm_cube_gbr_asb = dhm_cube_gbr_tmp.aggregated_by(['year'], iris.analysis.SUM)
    dhm_cube_gbr_asb_tmp = dhm_cube_gbr_asb.data
    dhm_cube_gbr_asb_tmp[np.where(dhm_cube_gbr_asb_tmp > 1.0)] = 1.0
    dhm_cube_gbr_asb.data = dhm_cube_gbr_asb_tmp
    return dhm_cube_gbr_asb
    
for model in models:
    dhm_cube_gbr = extract_region(cube_dict[model]['dhw_midnight'],lon_west,lon_east,lat_south,lat_north)
#     dhm_cube_gbr_Spillman_2013 = extract_region(cube_dict[model]['dhm_cube_Spillman_2013'],lon_west,lon_east,lat_south,lat_north)
#     dhm_cube_gbr_tmp = dhm_cube_gbr_Spillman_2013.copy()
    dhm_cube_gbr_asb = asb(dhm_cube_gbr,2.0)
    dhm_cube_gbr_asb_area_avg = area_avg(dhm_cube_gbr_asb)
    cube_dict[model]['asb_midnight'] = dhm_cube_gbr_asb
    cube_dict[model]['asb_avg_midnight'] = dhm_cube_gbr_asb_area_avg
    cube_dict[model]['dhw_avg_midnight'] = area_avg(dhm_cube_gbr.aggregated_by(['year'], iris.analysis.MEAN))
    dhm_cube_gbr = extract_region(cube_dict[model]['dhw_daily_avg'],lon_west,lon_east,lat_south,lat_north)
#     dhm_cube_gbr_Spillman_2013 = extract_region(cube_dict[model]['dhm_cube_Spillman_2013'],lon_west,lon_east,lat_south,lat_north)
#     dhm_cube_gbr_tmp = dhm_cube_gbr_Spillman_2013.copy()
    dhm_cube_gbr_asb = asb(dhm_cube_gbr,2.0)
    dhm_cube_gbr_asb_area_avg = area_avg(dhm_cube_gbr_asb)
    cube_dict[model]['asb_daily_avg'] = dhm_cube_gbr_asb
    cube_dict[model]['asb_avg_daily_avg'] = dhm_cube_gbr_asb_area_avg
    cube_dict[model]['dhw_avg_daily_avg'] = area_avg(dhm_cube_gbr.aggregated_by(['year'], iris.analysis.MEAN))
#     cube_dict[model]['dhw_cube_Spillman_2013_avg'] = area_avg(dhm_cube_gbr_Spillman_2013.aggregated_by(['year'], iris.analysis.MEAN))

In [ ]:
plt.figure(num=None, figsize=(12, 12), dpi=80, facecolor='w', edgecolor='k')


for model in models:
#     print cube_dict[model]['asb'].coord('year').points
#     y = cube_dict[model]['asb_avg'].data
#     y[np.where(y > 0.0)] = 1
    plt.scatter(cube_dict[model]['asb_daily_avg'].coord('year').points,cube_dict[model]['asb_avg_daily_avg'].data,alpha=0.5,color=model_dict[model]['color'])
    plt.scatter(cube_dict[model]['asb_midnight'].coord('year').points,cube_dict[model]['asb_avg_midnight'].data,alpha=0.5,color=model_dict[model]['color'],marker= '+',s=250,label = model)

# plt.ylim([-0.1,1.1])
plt.legend()
plt.show()

In [ ]:
plt.figure(num=None, figsize=(8, 8), dpi=80, facecolor='w', edgecolor='k')

# for model in models:
#     qplt.plot(cube_dict[model]['dhm_cube_Spillman_2013_avg'],lw = 2,alpha=0.5,color=model_dict[model]['color'])

    
for model in models:
    qplt.plot(cube_dict[model]['dhw_avg_daily_avg'],lw=2,color=model_dict[model]['color'],label = model,alpha=0.5)
    qplt.plot(cube_dict[model]['dhw_avg_midnight'],'--',lw=2,color=model_dict[model]['color'],label = model,alpha=0.5)


# plt.ylim([0,0.5])
plt.legend()

In [ ]:
note also that I'm downloading the crw sst satilite data on my desktop at moment to batmobile obs